In [ ]:
import json
import sys
import torch
from transformers import BertTokenizer
from bert_score import score as bert_score

# Initialize BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Function to calculate BERT score between user answer and correct answer
def calculate_bert_score(user_answer, correct_answer):
    P, R, F1 = bert_score([user_answer], [correct_answer], lang='en', model_type='bert-base-uncased')
    return F1.item()

# Function to check answer
def check_answer(question, user_answer, correct_answers):
    max_total_score = 0
    for correct_answer in correct_answers:
        correct_answer_lines = break_at_full_stop(correct_answer)
        max_score = -1
        for correct_answer_line in correct_answer_lines:
            bertscore_val = calculate_bert_score(user_answer, correct_answer_line)
            total_score = 1 * bertscore_val 
            max_score = max(max_score, total_score)
        max_total_score = max(max_score, max_total_score)
    return max_total_score

# Break text into lines at full stops
def break_at_full_stop(text):
    lines = text.split('.')
    # Remove any empty strings resulting from consecutive full stops
    lines = [line.strip() for line in lines if line.strip()]
    return lines

# Load input data
def load_input():
    input_data = json.loads(sys.stdin.read())
    return input_data

# Main function to handle input and calculate score
def main():
    input_data = load_input()
    question = input_data['question']
    user_answer = input_data['user_answer']
    correct_answers = input_data['correct_answers']
    
    user_answer_lines = break_at_full_stop(user_answer)
    maxscore = 0
    for user_answer_line in user_answer_lines:
        score = check_answer(question, user_answer_line, correct_answers)
        maxscore += score
    
    maxscore = maxscore / len(user_answer_lines)
    score_percentage = (-45 + 100 * maxscore) / 0.55
    
    output_data = {
        'score': score_percentage
    }
    
    # Output the result
    print(json.dumps(output_data))

if __name__ == "__main__":
    main()
